In [ ]:
import landaupy
from landaupy import landau
import matplotlib.pyplot as plt
import numpy as np
from random import gauss

In [47]:
#Constants:

c=3*10**8 #Speed of light in m.s^{-1}
me=5.11*10**-4 #Mass of the electron in GeV/c²

#Parameters:

q=0.3 #for a particle with a charge 1.6*10**-19 C
e=1.6*10**-19 #Elementary charge

In [68]:
"""
Pixel Firing Threshold
"""

#Initial position:

z0=0.1
y0=0.1

#Initial momentum in GeV/c:

p_z0=-3
p_y0=0

#Initial velocities in c:

v_z0=p_z0/me
v_y0=p_y0/me

#constants

th=14e-4 #detector thickness (in m)
rho=2.2 #density of detector (in g/cm3)
x=rho*th #detector thickness (in g/cm2)
ze=14 #Number of electrons of the detector's atom
j=0.200 #density effect correction
threshold=100 #environ 100e-
ratio_ZA=0.49930
Wi=3.6 #minimum energy required to create an electron-hole pair for Si (in eV)
K=0.307 #variable given by 4 * pi * Na * re^2 * me * c^2 where Na = Avogadro number, re = electron radius, me = electron mass
        #value given by PDG

#Parameters related to the Landau distribution:
ksi=(K/2)*ratio_ZA*(x/(beta**2))
w=4.018*ksi #width of Landau distribution

#we implement Landau distribution

def most_probable_value(Z):
    #Needed parameters:
    v0=np.sqrt((p_z0)**2+(p_y0)**2)
    beta=v0/c
    g=np.sqrt(1-beta**2) #inverse Lorentz coefficient
    
    #Parameters related to the Landau distribution:
    ksi=(0.1535*ze**2*rho)*ratio_ZA*(x/(beta**2)) #Parameter of the Landau distribution
    w=4.018*ksi #width of Landau distribution
    
    #Computation of the 'I' parameter:
    if Z<13:
        I=12+7/Z #average excitation energy
    else:
        I=9.76+58.8*(Z**(-1.19))
        
    #Most Probable Value formula:
    delta_p=ksi*(np.log((2*me*10**(9)*((beta)/g)**2)/I)+np.log(ksi/I)+j-beta**2) #most probable value
    
    return(delta_p)
    
delta_p_si=most_probable_value(14) #Z=14 for silicium

#we want nb of electron hole pair = energy deposited/Wi
#creates 100 random values from the Landau distribution

X=np.linspace(delta_p_si-1E8,delta_p_si+1E8,100000)
pdf = landau.pdf(X, delta_p_si, w)

samples = landau.sample(x_mpv=delta_p_si, xi=w, n_samples=9999)
#x_mpv is the most probable energy of Landau distribution
#xi is the width

def list_paires(sample,seuil):
    list=[]
    for i in range(0,len(sample)):
        list.append(sample[i]//seuil)
    return(list)
  
#the function takes as argument a random list of energies according to the landau distribution
#and the electron hole pair creation threshold, it returns a list containing the number of times
#that we have the threshold in each of the energy values in the list
#(Euclidean division of the energy deposited by the threshold energy)

def compte_pixel(list_paires,threshold):
    n=0
    for i in range(0,len(list_paires)):
        if list_paires[i]>=threshold:
            n+=1
    return(n)

#this function takes as argument the return of list_pairs and the minimum energy needed to turn on the pixel
#it returns the number of particles with enough energy to light a pixel

pixel = compte_pixel(samples,Wi)
print("number total of pixel:", len(samples),"number of lit pixel", pixel)

number total of pixel: 9999 number of lit pixel 9229


/srv/conda/envs/notebook/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in sqrt
  
